# Setting things up

## About this notebook

In this notebook, we embark on a journey to elevate the predictive capabilities of the California Housing Prices Dataset through advanced preprocessing using the ``ContinuousCarver`` pipeline. Renowned for its association-maximizing discretization, ``ContinuousCarver`` is a powerful Python tool designed to handle diverse data types—whether they be quantitative or qualitative. Our specific goal is to prepare the dataset for continuous regression tasks, such as predicting housing prices.

The California Housing Prices Dataset is a treasure trove of features, encompassing information on factors like square footage, bedrooms, location, and more. By employing ``ContinuousCarver``, we aim to seamlessly discretize both quantitative and qualitative features, tailoring them for optimal representation in our continuous regression models.

Throughout this notebook, we'll explore the intricacies of ``ContinuousCarver``'s discretization pipeline, witnessing its adaptability to a variety of data types. Whether it involves transforming square footage or encoding location information, ``ContinuousCarver`` ensures that each feature is finely tuned for our regression tasks.

Join us in this exploration as we leverage the power of ``ContinuousCarver`` to preprocess the California Housing Prices Dataset. Through effective feature engineering and discretization, our aim is to create a dataset that captures the nuanced relationships within the housing market, setting the stage for the development of accurate and impactful continuous regression models.

Let's dive in and uncover the potential of ``ContinuousCarver`` in transforming the California Housing Prices Dataset for optimal predictive modeling.


## Installation

In [1]:
# %pip install AutoCarver[jupyter]

## Califorinia Housing Prices Data

In this example notebook, we will use the California Housing Prices dataset.

The California Housing Prices dataset is a well-known dataset in the field of machine learning and statistics. It provides information about housing districts in California and is frequently used for regression analysis and predictive modeling tasks.

Comprising housing-related metrics for various districts in California, such as median house value, median income, housing median age, average rooms, average bedrooms, population, households, and more, the California Housing Prices dataset is a valuable resource for exploring the relationships between different features and predicting the median house values (continuous regression).

In [2]:
from sklearn import datasets

# Load dataset directly from sklearn
housing = datasets.fetch_california_housing(as_frame=True)

# conversion to pandas
housing_data = housing["data"]
housing_data[housing["target_names"][0]] = housing["target"]

# Display the first few rows of the dataset
housing_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Target type and Carver selection

In [3]:
target = "MedHouseVal"

housing_data[target].describe()

count    20640.000000
mean         2.068558
std          1.153956
min          0.149990
25%          1.196000
50%          1.797000
75%          2.647250
max          5.000010
Name: MedHouseVal, dtype: float64

The target ``"MedHouseVal"`` is a continuous target of type ``float64`` used in a regression task. Hence we will use ``AutoCarver.ContinuousCarver`` and ``AutoCarver.selectors.RegressionSelector`` in following code blocks.

## Data Sampling

In [4]:
from sklearn.model_selection import train_test_split

# stratified sampling by target
train_set, dev_set = train_test_split(housing_data, test_size=0.33, random_state=42)

# checking target rate per dataset
train_set[target].mean(), dev_set[target].mean()

(np.float64(2.0666362048018514), np.float64(2.072459655020552))

## Picking up columns to Carve

In [5]:
train_set.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
5088,0.9809,19.0,3.187726,1.129964,726.0,2.620939,33.98,-118.28,1.214
17096,4.2232,33.0,6.189696,1.086651,1015.0,2.377049,37.46,-122.23,3.637
5617,3.5488,42.0,4.821577,1.095436,1044.0,4.331950,33.79,-118.26,2.056
20060,1.6469,24.0,4.274194,1.048387,1686.0,4.532258,35.87,-119.26,0.476
895,3.9909,14.0,4.608303,1.089350,2738.0,2.471119,37.54,-121.96,2.360


In [6]:
# column data types
train_set.dtypes

MedInc         float64
HouseAge       float64
AveRooms       float64
AveBedrms      float64
Population     float64
AveOccup       float64
Latitude       float64
Longitude      float64
MedHouseVal    float64
dtype: object

All features are quantitative continuous features at the exception of ``Latitude`` and ``Longitude`` which are geographical featues (not supported by ``AutoCarver`` as is). All other features will be added to the list of ``quantitative_features``.

In [7]:
from AutoCarver import Features

# lists of features per data type
features = Features(quantitatives=["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup"])

C:\Users\defra\Desktop\git\PROJECTS\AutoCarver\AutoCarver\combinations\utils\combination_evaluator.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Using AutoCarver

## AutoCarver settings

### Representativness of modalities

The attribute ``min_freq`` allows one to choose the minimum frequency per basic modalities. It is used:

- For quantitative features, to define the number of quantiles to initialy discretize the features with.

- For qualitative features, to define the threshold under which a modality is grouped to either a default value or its closest modality.

In [8]:
min_freq = 0.1

**Tip:** should be set between ``0.01`` (slower, preciser, less robust) and ``0.2`` (faster, more robust)

### Desired number of modalities

The attribute ``max_n_mod`` allows one to choose the maximum number of modalities per carved feature. It is used by **Carvers** has the upper limit of number of modalities per consecutive combination of modalities.

In [9]:
max_n_mod = 4

**Tip:** should be set between ``3`` (faster, more robust) and ``7`` (slower, preciser, less robust)

### Grouping NaNs

The attribute ``dropna`` allows one to choose whether or not ``nan`` should be grouped with another modality. If set to ``True``, **Carvers** will first find the most suitable combination of non-``nan`` values, and then test out all possible combinations with ``nan``.

In [10]:
dropna = False  # anyway, there are no numpy.nan in this dataset

#### Type of output carved features

The attribute ``ordinal_encoding`` allows one to choose the output type:

* Use ``True`` for integer output of ranked modalities (default)
* Use ``False`` for string output of modalities

In [11]:
ordinal_encoding = True

## Fitting AutoCarver

* First, all quantitative features are discretized:
    1. Using ``ContinuousDiscretizer`` for quantile discretization that keeps track of over-represented values (more frequent than ``min_freq``)
    2. Using ``OrdinalDiscretizer`` for any remaining under-represented values (less frequent than ``min_freq/2``) to be grouped with its closest modality

* Second, all features are carved following this recipe, for all classes of ``train_set[target]`` (except one):
    1. The raw distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the discretization step
    2. Grouping modalities: all consecutive combinations of modalities are applied to ``train_set``
    3. Computing associations: the association metric (Tschruprow's T, by default) is computed with the provided ``train_set[target]``
    4. Combinations are sorted in descending order by association value
    5. Testing robustness: finds the first combination that checks the following:
        - Representativness of modalities on ``train_set`` and ``dev_set`` (all should be more frequent than ``min_freq/2``)
        - Distinct target rates per consecutive modalities on ``train_set`` and ``dev_set`` 
        - No inversion of target rates between ``train_set`` and ``dev_set`` (same ordering of modalities by target rate)
    6. (Optional) If requested via ``dropna=True``, and if any, all combinations of modalities with ``nan`` are applied to ``train_set`` and steps 3. and 4. are run
    7. The carved distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the carving step

In [12]:
from AutoCarver import ContinuousCarver

# intiating AutoCarver
auto_carver = ContinuousCarver(
    features=features,
    min_freq=min_freq,
    max_n_mod=max_n_mod,
    dropna=dropna,
    ordinal_encoding=ordinal_encoding,
    verbose=True,  # showing statistics
    copy=True,  # whether or not to return a copy of the input dataset
)

# fitting on training sample, a dev sample can be specified to evaluate carving robustness
train_set_processed = auto_carver.fit_transform(train_set, train_set[target], X_dev=dev_set, y_dev=dev_set[target])

------
--- [QuantitativeDiscretizer] Fit Features(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup'])
 - [ContinuousDiscretizer] Fit Features(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup'])
 - [OrdinalDiscretizer] Fit Features(['HouseAge'])
------

---------
------ [ContinuousCarver] Fit Features(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup'])
--- [ContinuousCarver] Fit Quantitative('MedInc') (1/6)
 [ContinuousCarver] Raw distribution


,target_rate,frequency
x <= 1.6e+00,1.1102,0.0500
1.6e+00 < x <= 1.9e+00,1.1285,0.0500
1.9e+00 < x <= 2.2e+00,1.2198,0.0500
2.2e+00 < x <= 2.4e+00,1.3171,0.0500
2.4e+00 < x <= 2.6e+00,1.3817,0.0500
2.6e+00 < x <= 2.7e+00,1.5409,0.0500
2.7e+00 < x <= 3.0e+00,1.6159,0.0500
3.0e+00 < x <= 3.1e+00,1.6906,0.0499
3.1e+00 < x <= 3.3e+00,1.8232,0.0500
3.3e+00 < x <= 3.5e+00,1.9059,0.0500


Testing robustness    :   0%|          | 0/1159 [00:00<?, ?it/s]




 [ContinuousCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 2.6e+00 
 1.2314 
 0.2500 
 
 
 2.6e+00 < x <= 4.0e+00 
 1.8016 
 0.3500 
 
 
 4.0e+00 < x <= 5.5e+00 
 2.3587 
 0.2499 
 
 
 5.5e+00 < x 
 3.5900 
 0.1501 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 1.2315 
 0.2554 
 
 
 1.8222 
 0.3509 
 
 
 2.3953 
 0.2446 
 
 
 3.5721 
 0.1491

...

--- [ContinuousCarver] Fit Quantitative('AveOccup') (6/6)
 [ContinuousCarver] Raw distribution


,target_rate,frequency
x <= 1.87e+00,2.7122,0.0500
1.87e+00 < x <= 2.07e+00,2.6633,0.0500
2.07e+00 < x <= 2.22e+00,2.3373,0.0500
2.22e+00 < x <= 2.34e+00,2.3080,0.0500
2.34e+00 < x <= 2.43e+00,2.1976,0.0500
2.43e+00 < x <= 2.51e+00,2.2064,0.0500
2.51e+00 < x <= 2.60e+00,2.1736,0.0500
2.60e+00 < x <= 2.67e+00,2.1862,0.0500
2.67e+00 < x <= 2.74e+00,2.1378,0.0500
2.74e+00 < x <= 2.82e+00,2.1902,0.0500


Testing robustness    :   0%|          | 3/1159 [00:00<00:06, 178.84it/s]




 [ContinuousCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 2.2e+00 
 2.5709 
 0.1501 
 
 
 2.2e+00 < x <= 3.1e+00 
 2.1681 
 0.5001 
 
 
 3.1e+00 < x <= 3.6e+00 
 1.8729 
 0.1998 
 
 
 3.6e+00 < x 
 1.4822 
 0.1501 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 2.5615 
 0.1461 
 
 
 2.1836 
 0.5129 
 
 
 1.8527 
 0.1869 
 
 
 1.5056 
 0.1541

## AutoCarver analysis

### Carving Summary

In [13]:
auto_carver.summary()

content  \
feature                    kruskal     n_mod label                             
Quantitative('MedInc')     6037.182135 4     0                  x <= 2.6e+00   
                                             1        2.6e+00 < x <= 4.0e+00   
                                             2        4.0e+00 < x <= 5.5e+00   
                                             3                   5.5e+00 < x   
Quantitative('HouseAge')   163.527841  4     0                  x <= 2.2e+01   
                                             1        2.2e+01 < x <= 2.6e+01   
                                             2        2.6e+01 < x <= 4.5e+01   
                                             3                   4.5e+01 < x   
Quantitative('AveRooms')   1391.586489 4     0                  x <= 5.2e+00   
                                             1        5.2e+00 < x <= 5.9e+00   
                                             2        5.9e+00 < x <= 6.5e+00   
                                             3                   6.5e+00 < x   
Quantitative('AveBedrms')  315.794350  4     0                 x <= 9.67e-01   
                                             1      9.67e-01 < x <= 1.06e+00   
                                             2      1.06e+00 < x <= 1.14e+00   
                                             3                  1.14e+00 < x   
Quantitative('Population') 16.109709   4     0                  x <= 6.3e+02   
                                             1        6.3e+02 < x <= 8.6e+02   
                                             2        8.6e+02 < x <= 2.2e+03   
                                             3                   2.2e+03 < x   
Quantitative('AveOccup')   991.408301  4     0                  x <= 2.2e+00   
                                             1        2.2e+00 < x <= 3.1e+00   
                                             2        3.1e+00 < x <= 3.6e+00   
                                             3                   3.6e+00 < x   

                                                    target_rate  frequency  
feature                    kruskal     n_mod label                          
Quantitative('MedInc')     6037.182135 4     0         1.231421   0.250000  
                                             1         1.801562   0.350014  
                                             2         2.358660   0.249928  
                                             3         3.590040   0.150058  
Quantitative('HouseAge')   163.527841  4     0         1.949361   0.348568  
                                             1         2.103173   0.100521  
                                             2         2.050927   0.443665  
                                             3         2.478542   0.107246  
Quantitative('AveRooms')   1391.586489 4     0         1.805255   0.500000  
                                             1         1.906098   0.199957  
                                             2         2.227531   0.149986  
                                             3         2.990676   0.150058  
Quantitative('AveBedrms')  315.794350  4     0         2.070937   0.100014  
                                             1         2.171033   0.450029  
                                             2         2.047547   0.299899  
                                             3         1.788831   0.150058  
Quantitative('Population') 16.109709   4     0         2.086394   0.150347  
                                             1         2.174297   0.149841  
                                             2         2.043255   0.549754  
                                             3         2.024995   0.150058  
Quantitative('AveOccup')   991.408301  4     0         2.570888   0.150058  
                                             1         2.168126   0.500072  
                                             2         1.872867   0.199812  
                                             3         1.482183

* As requested with ``ordinal_encoding=True``, output labels are integers of modalities

* For quantitative feature ``Population``, the selected combination of modalities groups populations as follows:
    * label ``0``: lower or equal to 630 people (``content="x <= 6.3e+02"``)
    * label ``1``: greater than 630 people and lower or equal to 860 people  (``content="6.3e+02 < x <= 8.6e+02"``)
    * label ``2``: greater than 860 people and lower or equal to 2200 people (``content="8.6e+02 < x <= 2.2e+03"``)
    * label ``3``: higher than 2200 people (``content="2.2e+03 < x"``)

### Detailed overview of tested combinations

In [14]:
features["AveOccup"].history.head(7)

,info,kruskal,combination,n_mod,dropna,train,viable,dev
0,Raw distribution (n_mod=20>max_n_mod=4),1062.072498,"{'x <= 1.87e+00': 'x <= 1.87e+00', '1.87e+00 <...",20,False,NaN,NaN,NaN
1,Not viable,994.514410,"{'x <= 1.87e+00': 'x <= 1.87e+00', '1.87e+00 <...",4,False,"{'viable': True, 'info': ''}",False,"{'viable': False, 'info': 'Non-representative ..."
2,Not viable,994.504665,"{'x <= 1.87e+00': 'x <= 1.87e+00', '1.87e+00 <...",4,False,"{'viable': True, 'info': ''}",False,"{'viable': False, 'info': 'Non-representative ..."
3,Not viable,991.504255,"{'x <= 1.87e+00': 'x <= 1.87e+00', '1.87e+00 <...",4,False,"{'viable': True, 'info': ''}",False,"{'viable': False, 'info': 'Non-representative ..."
4,Best for kruskal and max_n_mod=4,991.408301,"{'x <= 1.87e+00': 'x <= 1.87e+00', '1.87e+00 <...",4,False,"{'viable': True, 'info': ''}",True,"{'viable': True, 'info': ''}"
5,Not checked,991.308986,"{'x <= 1.87e+00': 'x <= 1.87e+00', '1.87e+00 <...",4,False,NaN,NaN,NaN
6,Not checked,988.666983,"{'x <= 1.87e+00': 'x <= 1.87e+00', '1.87e+00 <...",4,False,NaN,NaN,NaN


In [15]:
features["AveOccup"].history.dev[1]

{'viable': False, 'info': 'Non-representative modality for min_freq=10.00%'}

* The most associated combination of feature ``AveOccup`` (the first tested out, where ``info!="Raw distribution"``) did not pass the viability tests. When looking in ``history.dev``:
    * ``"Non-representative modality for min_freq=10.00%"``: tells us that a modality is unstable between ``train_set`` and ``dev_set``

* For feature feature ``AveOccup``, the 4th combination is the first to pass tests:
    - ``info="Best for kruskal and max_n_mod=4"``
    - Kruskal-Wallis' H with ``MedHouseVal`` is ``991.408301`` for this combination
    - Following combinations (less associated with the target) where not tested: ``info="Not checked"``

* For all combinations ``dropna=False`` means that it is not a combination in which ``nan``s are being grouped with other modalities (as requested with ``dropna=False``)

## Saving and Loading AutoCarver

### Saving

All **Carvers** can safely be stored as a .json file.

In [16]:
auto_carver.save("continuous_carver.json")

### Loading

**Carvers** can safely be loaded from a .json file.

In [17]:
from AutoCarver import ContinuousCarver

# loading json file
auto_carver = ContinuousCarver.load("continuous_carver.json")

## Applying AutoCarver

In [18]:
dev_set_processed = auto_carver.transform(dev_set)

In [19]:
dev_set_processed[auto_carver.features].apply(lambda u: u.value_counts(dropna=False, normalize=True))

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
0.0,0.255432,0.340282,0.502789,0.106577,0.157223,0.146066
1.0,0.350851,0.105843,0.203318,0.451703,0.153553,0.512918
2.0,0.244568,0.447005,0.144011,0.295508,0.539049,0.186876
3.0,0.149149,0.106870,0.149883,0.146213,0.150176,0.154140


# Updating/Adjusting AutoCarver
## Updating group thresholds/values

Let's say one wants to adjust the values of a carved quantitative feature for better readability. Take feature ``"Population"`` for example, one could prefer group thresholds rounded to the nearest ten.

In [24]:
feature = "Population"
thresholds = auto_carver.values_orders[feature]
print(f"Upper thresholds for groups of feature {feature}: {thresholds}")

Upper thresholds for groups of feature Population: [353.0, 793.0, 864.0, 2152.0, inf]


In [25]:
# grouping values into closest rounded hundred
for threshold in thresholds[:-1]:  # be carefull not to round the np.inf and to keep it whatsoever
    rounded_threshold = 10 * round(threshold / 10, 0)
    auto_carver.update_discretizer(feature, "replace", threshold, rounded_threshold)

auto_carver.summary(feature)

label                       content
feature    dtype                                     
Population float      0              [x <= 3.500e+02]
           float      1  [3.500e+02 < x <= 7.900e+02]
           float      2  [7.900e+02 < x <= 8.600e+02]
           float      3  [8.600e+02 < x <= 2.150e+03]
           float      4               [2.150e+03 < x]

Using ``ContinuousCarver.update_discretizer()`` we managed to round the thresholds of our groups. Be careful with changes to **Carvers** as no viability checks are performed.

## Grouping built modalities

Let's say one thinks there are to many modalities for carved feature ``"Population"``. Using ``ContinuousCarver.update_discretizer()`` we can easily group two existing groups.

In [26]:
feature = "Population"
thresholds = auto_carver.values_orders[feature]
print(f"Upper thresholds for groups of feature {feature}: {thresholds}")

Upper thresholds for groups of feature Population: [350.0, 790.0, 860.0, 2150.0, inf]


Threshold values ``790`` and ``860`` seem quite close compared to ``350`` and ``2150``. Let's group them:

In [27]:
auto_carver.update_discretizer(
    feature, "group", 790, 860
)  # be careful to respect the real number ordering, otherwise labels and discretizations will be wrong
auto_carver.summary(feature)

label                       content
feature    dtype                                     
Population float      0              [x <= 3.500e+02]
           float      1  [3.500e+02 < x <= 8.600e+02]
           float      2  [8.600e+02 < x <= 2.150e+03]
           float      3               [2.150e+03 < x]

## Adding new/unexpected modalities

Let's say a new modality or missing values appear when using **Carvers** on new data. One can easily add these modalities to an existing group of its choice, once again, using ``ContinuousCarver.update_discretizer()``.

In [28]:
feature = "Population"
thresholds = auto_carver.values_orders[feature]
print(f"Group thresholds for feature {feature}: {thresholds}")

Group thresholds for feature Population: [350.0, 860.0, 2150.0, inf]


Let's add missing values ``nan`` to the highest group (upper threshold at ``inf``):

In [29]:
from numpy import inf, nan

auto_carver.update_discretizer(
    feature, "group", nan, inf
)  # be careful to respect the real number ordering, otherwise labels and discretizations will be wrong
auto_carver.summary(feature)

label                       content
feature    dtype                                     
Population float      0              [x <= 3.500e+02]
           float      1  [3.500e+02 < x <= 8.600e+02]
           float      2  [8.600e+02 < x <= 2.150e+03]
           float      3      [2.150e+03 < x, __NAN__]

``nan`` has been added to the highest modality has the default missing value in **Carvers** (``str_nan="__NAN__"``). For the sake of the example, we will simulate some randomly missing values in the feature and apply the ``ContinuousCarver`` discretization:

In [30]:
# inserting missing values
dev_set_with_nan = dev_set.copy()
dev_set_with_nan.loc[dev_set_with_nan.sample(frac=0.1).index, feature] = nan

In [31]:
# processing datasets
dev_set_processed = auto_carver.transform(dev_set)
dev_set_with_nan_processed = auto_carver.transform(dev_set_with_nan)

In [32]:
print(
    "Distribution of Population that has no missing values (raw dev_set)\n",
    dev_set_processed[feature].value_counts(dropna=False, normalize=True).sort_index(),
)
print(
    "Distribution of Population that has missing values (dev_set_with_nan)\n",
    dev_set_with_nan_processed[feature].value_counts(dropna=False, normalize=True).sort_index(),
)

Distribution of Population that has no missing values (raw dev_set)
 Population
0.0    0.052554
1.0    0.254844
2.0    0.542278
3.0    0.150323
Name: proportion, dtype: float64
Distribution of Population that has missing values (dev_set_with_nan)
 Population
0.0    0.047123
1.0    0.230916
2.0    0.489430
3.0    0.232531
Name: proportion, dtype: float64


As requested, the missing values have been grouped with the modality labeled ``3``!

# Feature Selection
## Selectors settings

### Features to select from

Here all features have been carved using ``ContinuousCarver``, hence all features are qualitative.

In [33]:
features = qualitative_features + quantitative_features + ordinal_features


### Number of features to select

The attribute ``n_best`` allows one to choose the number of features to be selected per data type (quantitative and qualitative).

In [34]:
n_best = 6  # here the number of features is low, ClassificationSelector will only be used to compute useful statistics

## Using Selectors

In [35]:
from AutoCarver.selectors import RegressionSelector

# select the most target associated qualitative features
feature_selector = RegressionSelector(
    qualitative_features=features,
    n_best=n_best,
    verbose=True,  # displays statistics
)
best_features = feature_selector.select(train_set_processed, train_set_processed[target])

------
[Selector] Selecting from qualitative features: ['MedInc', 'AveBedrms', 'Population', 'AveRooms', 'HouseAge', 'AveOccup']
---

 - [Selector] Association between X and y


,dtype,pct_nan,pct_mode,mode,kruskal_measure
MedInc,float64,0.0000,0.2500,0.0000,6207.6768
AveRooms,float64,0.0000,0.6000,0.0000,1417.9360
AveOccup,float64,0.0000,0.4500,1.0000,1026.3004
AveBedrms,float64,0.0000,0.5500,0.0000,346.0749
HouseAge,float64,0.0000,0.3486,0.0000,164.2102
Population,float64,0.0000,0.5498,3.0000,29.0503



 - [Selector] Association between X and y, filtered for inter-feature assocation


,dtype,pct_nan,pct_mode,mode,kruskal_measure
MedInc,float64,0.0000,0.2500,0.0000,6207.6768
AveRooms,float64,0.0000,0.6000,0.0000,1417.9360
AveOccup,float64,0.0000,0.4500,1.0000,1026.3004
AveBedrms,float64,0.0000,0.5500,0.0000,346.0749
HouseAge,float64,0.0000,0.3486,0.0000,164.2102
Population,float64,0.0000,0.5498,3.0000,29.0503



 - [Selector] Selected qualitative features: ['MedInc', 'AveRooms', 'AveOccup', 'AveBedrms', 'HouseAge', 'Population']
------



* Feature ``MedInc`` is the most associated with the target ``MedHouseVal``:
    - Kruskal-Wallis' H value is ``kruskal_measure=6207.67678``
    - It has 0 % of NaNs (``pct_nan=0.0``) 
    - Its mode, ``0``, represents 25 % of observed data (``pct_nan=0.2500``)

* Here, no feature where filtered out for there inter-feature association or over-represented values (no thresholds were set)

## What's next?

* Thanks to **Carvers** all of your features are now optimally processed for your regression task!
* As a final step towards your model, **Selectors** can prove to be handy tools to operate target optimal Data Pre-Selection, so make sure to check out [Selectors Examples](https://autocarver.readthedocs.io/en/latest/selectors_examples.html)!

## Well done!

Your commitment to achieving optimal results in continuous regression tasks shines through in your meticulous use of **AutoCarver**'s ``ContinuousCarver`` for data preprocessing. By fine-tuning and optimizing your dataset, you have set the stage for robust and accurate machine learning models.

The ``ContinuousCarver`` has proven to be a valuable ally in your pursuit of excellence, carving out a path toward enhanced feature representation and model interpretability. Your dedication to refining the data preprocessing steps reflects a commitment to extracting the maximum value from your datasets.

We extend our sincere appreciation for choosing **AutoCarver** as your companion in the data preprocessing journey. Your use of **AutoCarver** demonstrates a dedication to leveraging cutting-edge tools for achieving excellence in continuous regression tasks.

As you transition to the modeling phase, may the carefully crafted features and preprocessing steps contribute to the success of your predictive models. We're excited to see the impact of your work and are grateful for the opportunity to be part of your data science endeavors.

Thank you for trusting **AutoCarver**, and we wish you continued success in your data-driven ventures.